# Staying up-to-date

Hello everyone! Today we will work with *Tensorflow callbacks* to send status notifications using **Telegram**.


* **Telegram** is a free messaging app that focuses on speed 
and security, with over 500 million monthly active users.

In [ ]:
import tensorflow as tf # Library for machine learning and AI 
import requests # Library used to send HTTP/1.1 requests

# Convolutional Neural Network

First, we need to create our cnn (see Exercise 4) to work later with our callbacks:

In [ ]:
mnist = tf.keras.datasets.mnist #loading mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()  #assigning the test and train data
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (3,3), input_shape=(28,28,1), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
cnn.compile("adam", "sparse_categorical_crossentropy", metrics=['acc'])

# Telegram bot

## Create a callback that works using Telegram.

1. ToDo: Search how to create a Bot using Telegram and get its personal token.

In [ ]:
personal_token = 'here comes the token'

2. ToDo: Create the Tensorflow callback that notifies using the Telegram bot.

In [ ]:
# 2.1 ToDo: Declare the subclass to create a custom callback
class botCallback(#something here):
  def __init__(self,personal_token):
    self.personal_token = personal_token
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/getUpdates'
    self.response = requests.get(self.ping_url).json()
    self.chat_id = self.response['result'][0]['message']['chat']['id']
    self.last_message_id = self.response['result'][-1]["message"]["message_id"]

  def send_message(self,message):
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/sendMessage?'+\
                    'chat_id='+str(self.chat_id)+\
                    '&parse_mode=Markdown'+\
                    '&text='+message
    self.response = requests.get(self.ping_url)

    #2.2 ToDo: Message notifying that it is starting to learn.

    #2.3 ToDo: Message notifying the results at the end of each epoch.

    #2.4 ToDo: Message notifying that you are done training.

    #2.5 ToDo: Message notifying the results of cnn.evaluate.

    #2.6 ToDo: Check the last message and if it is "stop" stop the training

  pass
#Note: The Callback has to do every task asked, otherwise you don't get the point.

Answer:

In [ ]:
class botCallback(tf.keras.callbacks.Callback):
  def __init__(self,personal_token):
    self.personal_token = personal_token
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/getUpdates'
    self.response = requests.get(self.ping_url).json()
    self.chat_id = self.response['result'][0]['message']['chat']['id']
    self.last_message_id = self.response['result'][-1]["message"]["message_id"]

  def send_message(self,message):
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/sendMessage?'+\
                    'chat_id='+str(self.chat_id)+\
                    '&parse_mode=Markdown'+\
                    '&text='+message
    self.response = requests.get(self.ping_url)
  
  def check_stop(self):
    self.ping_url = 'https://api.telegram.org/bot'+str(self.personal_token)+'/getUpdates'
    self.response = requests.get(self.ping_url).json()
    self.msg = self.response['result'][-1]["message"]["text"]
    self.message_id = self.response['result'][-1]["message"]["message_id"]
    if self.message_id != self.last_message_id and self.msg == 'stop':
      self.model.stop_training = True
      self.send_message("You stopped the training!")
    else: 
      self.model.stop_training = None
    self.last_message_id = self.message_id
    


  def on_train_begin(self, logs=None):
    self.send_message("I'm starting to train!")

  def on_epoch_end(self, epoch, epoch_logs):
    message = ' epoch {}\n Training Accuracy : {:7.3f}\n Training Loss : {:7.3f}\n'.format(epoch+1,epoch_logs['acc'],epoch_logs['loss'])
    self.send_message(message)
    self.check_stop()

  # def on_train_batch_end(self, batch, logs = None):
  #   self.check_stop()


  def on_train_end(self, logs=None):
    self.send_message("I'm done training!")

  def on_test_end(self,logs=None):
    self.send_message('Test results: Accuracy of {:7.3f}% '.format(logs['acc']*100))

We assign the callback to a variable called ```bot_callback```:

In [ ]:
bot_callback = botCallback(personal_token)

3. ToDo: make the callback work on both train and test:

In [ ]:
h = cnn.fit(x_train[..., None], y_train, epochs=10, callbacks = bot_callback)

In [ ]:
cnn.evaluate(x_test, y_test, callbacks = bot_callback)